In [2]:
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange

# Step1: prepare data.
#url = 'http://mattmahoney.net/dc/'
#
#def download(filename):
#    print('Starting downloading data ...')
#    filename, _ = urllib.request.urlretrieve(url + filename, filename)
#    print('Data downloaded!')
#    return filename
#
#def maybe_download(filename, expected_bytes):
#    """Download a file if not present or size is incorrect."""
#    if not os.path.exists(filename):
#        filename = download(filename)
#    else:
#        if os.stat(filename).st_size != expected_bytes:
#            os.remove(filename)
#            filename = download(filename)
#    
#    statinfo = os.stat(filename)
#    if statinfo.st_size == expected_bytes:
#        print('Found and verified', filename)
#    else:
#        print(statinfo.st_size)
#        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')        
#    return filename
#
#filename = maybe_download('text8.zip', 31344016)
#print("Data prepared!")
```
text下好了 上面可以注释掉
```
# Step2: split words.
filename ='./text8.zip'
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = f.read(f.namelist()[0]).split()
    return data

words = read_data(filename)
print('Data size', len(words))
```
```
# Step3: Build dictionary.
vocabulary_size = 50000
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0    # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words    # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

# Step4: function to generate training batch
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1    # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window    # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

print(generate_batch(4, 2, 1))
print(generate_batch(6, 3, 2))


# Step5: Build graph
import tensorflow as tf

# Training input data.
batch_size = 128
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

# Validate input data.
valid_size = 8        # Random set of words to evaluate similarity on.
valid_window = 100    # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# Look up embeddings for inputs.
embedding_size = 128    # Dimension of the embedding vector.
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

# Construct the variables for the NCE loss
nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
num_sampled = 64        # Number of negative examples to sample.
loss = tf.reduce_mean(
        tf.nn.nce_loss(nce_weights, nce_biases, train_labels,embed,num_sampled, vocabulary_size))
#loss = tf.reduce_mean(tf.nn.nce_loss(
#        nce_weights, nce_biases, embed, train_labels, num_sampled, vocabulary_size))

# Construct the SGD optimizer using a learning rate of 1.0.
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

# Compute the cosine similarity between minibatch examples and all embeddings.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

sess = tf.InteractiveSession()
tf.initialize_all_variables().run()

# Output evaluation result
def print_eval_result():
    sim = similarity.eval()
    for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
            close_word = reverse_dictionary[nearest[k]]
            log_str = "%s %s," % (log_str, close_word)
        print(log_str) 

print_eval_result()

# Step6: train model.
num_steps = 100001
average_loss = 0
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.
for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
    feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 5000 == 0:
        if step > 0: average_loss /= 5000
        # The average loss is an estimate of the loss over the last 2000 batches.
        print("Average loss at step ", step, ": ", average_loss)
        average_loss = 0

    if step % 20000 == 0:
        print("Validate evaluation result at at step ", step)
        print_eval_result()

final_embeddings = normalized_embeddings.eval()
sess.close()

# Step 7: Visualize the embeddings.
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    print(low_dim_embs.shape)
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
    plt.show()  
    plt.savefig(filename)

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels)

except ImportError:
    print("Please install sklearn and matplotlib to visualize embeddings.")

SyntaxError: invalid syntax (<ipython-input-2-b746d4303351>, line 39)

## 运行结果
Nearest to b'that': b'net', b'abscess', b'apprenticed', b'aroma', b'steel', b'linked', b'linda', b'overland',  
Nearest to b'only': b'valet', b'octavius', b'reverence', b'bundy', b'mixing', b'shaky', b'seasons', b'shula',  
Nearest to b'five': b'vhdl', b'aonb', b'pipelined', b'tacoma', b'cartooning', b'weyl', b'periodicity', b'buckner',  
Nearest to UNK: b'ambushes', b'departs', b'anna', b'stoiber', b'gael', b'loup', b'kabir', b'fabulous',  
Nearest to b'nine': b'quantized', b'thundering', b'scripting', b'frederick', b'cent', b'meditative', b'discharged', b'implosion',  
Nearest to b'on': b'toxicity', b'tabled', b'telling', b'kowtow', b'broncos', b'iucn', b'plateaus', b'colosseum',  
Nearest to b'history': b'wiesbaden', b'critical', b'sportsmanship', b'reuptake', b'guang', b'waterfront', b'capability', b'eruption',  
Nearest to b'no': b'bored', b'hurst', b'onion', b'soufriere', b'q', b'tn', b'insuring', b'zeppelins',  
Average loss at step  0 :  272.612884521  
Validate evaluation result at at step  0  
Nearest to b'that': b'net', b'abscess', b'apprenticed', b'aroma', b'steel', b'linda', b'linked', b'overland',  
Nearest to b'only': b'valet', b'octavius', b'reverence', b'bundy', b'mixing', b'shaky', b'seasons', b'shula',  
Nearest to b'five': b'vhdl', b'aonb', b'pipelined', b'tacoma', b'cartooning', b'weyl', b'periodicity', b'buckner',  
Nearest to UNK: b'ambushes', b'departs', b'anna', b'stoiber', b'gael', b'loup', b'kabir', b'fabulous',  
Nearest to b'nine': b'quantized', b'thundering', b'scripting', b'frederick', b'cent', b'meditative', b'discharged', b'implosion',  
Nearest to b'on': b'toxicity', b'tabled', b'telling', b'kowtow', b'broncos', b'iucn', b'plateaus', b'colosseum',  
Nearest to b'history': b'wiesbaden', b'critical', b'sportsmanship', b'reuptake', b'guang', b'waterfront', b'capability', b'eruption',  
Nearest to b'no': b'bored', b'hurst', b'onion', b'soufriere', b'q', b'tn', b'insuring', b'zeppelins',  
Average loss at step  5000 :  73.7759035952  
Average loss at step  10000 :  22.5834041986  
Average loss at step  15000 :  12.518386869  
Average loss at step  20000 :  8.44408598104  
Validate evaluation result at at step  20000  
Nearest to b'that': b'acacia', b'also', b'but', b'which', b'capitalism', b'hbox', b'and', b'this',  
Nearest to b'only': b'reverence', b'mathbf', b'winning', b'octavius', b'shaky', UNK, b'basins', b'seasons',  
Nearest to b'five': b'zero', b'nine', b'six', b'seven', b'eight', b'four', b'two', b'three',  
Nearest to UNK: b'dasyprocta', b'coke', b'vs', b'reginae', b'operatorname', b'amino', b'mathbf', b'agave',  
Nearest to b'nine': b'eight', b'six', b'seven', b'zero', b'five', b'four', b'three', b'agouti',  
Nearest to b'on': b'in', b'with', b'by', b'at', b'psi', b'for', b'reincarnation', b'and',  
Nearest to b'history': b'critical', b'capability', b'gland', b'museum', b'clement', b'agha', b'one', b'creation',  
Nearest to b'no': b'a', b'who', b'onion', b'circ', b'situ', b'category', b'amino', b'there',  
Average loss at step  25000 :  7.02860466447  
Average loss at step  30000 :  6.24210170503  
Average loss at step  35000 :  5.88875478163  
Average loss at step  40000 :  5.42356576786  
Validate evaluation result at at step  40000  
Nearest to b'that': b'which', b'this', b'acacia', b'but', b'it', b'one', b'also', b'hbox',  
Nearest to b'only': b'mathbf', b'reverence', b'boyer', b'winning', b'but', b'octavius', b'reginae', b'clark',  
Nearest to b'five': b'six', b'four', b'seven', b'eight', b'three', b'zero', b'two', b'nine',  
Nearest to UNK: b'dasyprocta', b'coke', b'and', b'agave', b'one', b'operatorname', b'amino', b'reginae',  
Nearest to b'nine': b'eight', b'seven', b'six', b'zero', b'five', b'four', b'three', b'agouti',  
Nearest to b'on': b'in', b'at', b'psi', b'with', b'from', b'by', b'callisto', b'two',  
Nearest to b'history': b'reuptake', b'recitative', b'gland', b'critical', b'museum', b'capability', b'agha', b'eruption',  
Nearest to b'no': b'a', b'there', b'situ', b'who', b'enol', b'onion', b'circ', b'some',  
Average loss at step  45000 :  5.29793579884  
Average loss at step  50000 :  5.11490167618  
Average loss at step  55000 :  5.12963704901  
Average loss at step  60000 :  5.0476238656  
Validate evaluation result at at step  60000  
Nearest to b'that': b'which', b'this', b'but', b'saguinus', b'acacia', b'also', b'tamarin', b'it',  
Nearest to b'only': b'but', b'boyer', b'tamarin', b'cebus', b'arctos', b'clark', b'under', b'saguinus',  
Nearest to b'five': b'four', b'six', b'three', b'eight', b'seven', b'zero', b'nine', b'two',  
Nearest to UNK: b'saguinus', b'tamarin', b'microcebus', b'dasyprocta', b'cebus', b'operatorname', b'michelob', b'perfective',  
Nearest to b'nine': b'eight', b'six', b'seven', b'four', b'five', b'zero', b'three', b'callithrix',  
Nearest to b'on': b'in', b'at', b'from', b'with', b'callisto', b'tamarin', b'psi', b'vacations',  
Nearest to b'history': b'reuptake', b'museum', b'guang', b'recitative', b'gland', b'wiesbaden', b'cebus', b'eruption',  
Nearest to b'no': b'a', b'there', b'situ', b'any', b'onion', b'some', b'circ', b'enol',  
Average loss at step  65000 :  4.83817691798  
Average loss at step  70000 :  4.84920379639  
Average loss at step  75000 :  4.81135193964  
Average loss at step  80000 :  4.79532247565  
Validate evaluation result at at step  80000  
Nearest to b'that': b'which', b'acacia', b'saguinus', b'but', b'this', b'however', b'tamarin', b'monteverdi',  
Nearest to b'only': b'but', b'iit', b'boyer', b'upanija', b'tamarin', b'shula', b'arctos', b'cebus',  
Nearest to b'five': b'six', b'four', b'seven', b'eight', b'three', b'nine', b'zero', b'two',  
Nearest to UNK: b'iit', b'saguinus', b'tamarin', b'microcebus', b'upanija', b'dasyprocta', b'thaler', b'cebus',  
Nearest to b'nine': b'eight', b'seven', b'six', b'five', b'four', b'zero', b'three', b'callithrix',  
Nearest to b'on': b'in', b'at', b'vacations', b'from', b'ueshiba', b'coprime', b'callisto', b'thaler',  
Nearest to b'history': b'guang', b'reuptake', b'museum', b'wiesbaden', b'eruption', b'recitative', b'vec', b'gland',  
Nearest to b'no': b'there', b'situ', b'any', b'onion', b'a', b'conclusive', b'circ', b'enol',  
Average loss at step  85000 :  4.78431438661  
Average loss at step  90000 :  4.70902515097  
Average loss at step  95000 :  4.67157746243  
Average loss at step  100000 :  4.6676021919  
Validate evaluation result at at step  100000  
Nearest to b'that': b'which', b'however', b'but', b'saguinus', b'acacia', b'this', b'what', b'aquilegia',  
Nearest to b'only': b'iit', b'tamarin', b'boyer', b'upanija', b'but', b'arctos', b'saguinus', b'thibetanus',  
Nearest to b'five': b'four', b'six', b'seven', b'eight', b'three', b'zero', b'two', b'nine',  
Nearest to UNK: b'tamarin', b'upanija', b'saguinus', b'iit', b'cebus', b'thaler', b'microcebus', b'mitral',  
Nearest to b'nine': b'eight', b'seven', b'six', b'five', b'four', b'zero', b'three', b'tamarin',  
Nearest to b'on': b'in', b'at', b'upon', b'from', b'tamarin', b'agave', b'callisto', b'ueshiba',  
Nearest to b'history': b'guang', b'museum', b'reuptake', b'wiesbaden', b'recitative', b'gland', b'eruption', b'cebus',  
Nearest to b'no': b'any', b'there', b'a', b'situ', b'onion', b'conclusive', b'circ', b'enol',  

![](https://github.com/xx674967/githubdesktop/blob/12/pic/12.4/1.png?raw=true)

## Tensorflow实现word2vec skip-garm模型
#### 不管是CBOW还是skip-gram，都可以使用Hierarchical softmax 或者Negative Sampling，Negative Sampling不再使用复杂的huffman树，这样可以大幅提高性能。
## 如何训练我们的神经网络
#### 假如我们有一个句子“The dog barked at the  mailman”。
1.首先我们选句子中间的一个词作为我们的输入词，例如我们选取“dog”作为input word

2.有了input word以后，我们再定义一个叫做skip_window的参数，它代表着我们从当前input word的一侧（左边或右边）选取词的数量。如果我们设置skip_window=2，那么我们最终获得窗口中的词（包括input word在内）就是['The', 'dog'，'barked', 'at']。skip_window=2代表着选取左input word左侧2个词和右侧2个词进入我们的窗口，所以整个窗口大小span=2x2=4。另一个参数叫num_skips，它代表着我们从整个窗口中选取多少个不同的词作为我们的output word，当skip_window=2，num_skips=2时，我们将会得到两组 (input word, output word) 形式的训练数据，即 ('dog', 'barked')，('dog', 'the')。

3.神经网络基于这些训练数据将会输出一个概率分布，这个概率代表着我们的词典中的每个词是output word的可能性。这句话有点绕，我们来看个栗子。第二步中我们在设置skip_window和num_skips=2的情况下获得了两组训练数据。假如我们先拿一组数据 ('dog', 'barked') 来训练神经网络，那么模型通过学习这个训练样本，会告诉我们词汇表中每个单词是“barked”的概率大小。

##